# COMP9318 Lab2

## Instructions
1. This note book contains instructions for **COMP9318-Lab2**.

* You are required to complete your implementation in a file `submission.py` provided along with this notebook.

* You are only allowed to use **Python 3.6** for implementation.

* You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures return by corresponding functions.

* You need to submit the code for **lab2** via following link: https://unswkg.net/submit/

* For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Piazza.

* If you choose to skip a question, leave the corresponding function body as it is (i.e., keep the `pass` line), otherwise it may affect your mark for other questions.

* You are allowed to add other functions and/or import additional modules (you may have to in this lab), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

* You should not import unnecessary modules/libraries, failing to import such modules at test time will lead to errors.

* We will provide immediate feedback on your submission. You can access your scores using the online submission portal on the same day. 

* For **Final Evaluation** we will be using a different dataset, so your final scores may vary.  

* You are allowed to submit as many times as you want before the deadline, but **ONLY the latest version will be kept and marked**.

* Submission deadline for this assignment is **20:59:59 on 23rd March, 2020 (Sydney Time)**. We will **not** accept any late submissions.

# Question 1: Optimized BUC algorithm (100 points)

You need to implement the full `buc_rec_optimized` algorithm with the single-tuple optimization (as described below). Given an input dataframe:

 A | B | M 
---|---|---
 1 | 2 | 100
 2 | 1 | 20

Invoking  `buc_rec_optimized` on this data will result in following dataframe: 


 A | B | M
---|---|---
 1 | 2 | 100
 1 |ALL| 100
 2 | 1 | 20 
 2 |ALL| 20
ALL| 1 | 20
ALL| 2 | 100
ALL|ALL| 120

We have pre-defined the function `buc_rec_optimized` in the file `submission.py`, and its helper functions are defined in the file `helper.py`. 

**Note:** You should use the functions defined in the file `helper.py`, you are not allowed to change this file. We will provide this file in the test environment.

## Input and output

Both input and output are dataframes.

The input dataframe (i.e., the base cuboid) is directly generated from the input file. Given the dimensionality of the base cuboid is $d$, each row is like:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 

The output dataframe contains $n$ rows, each for a non-empty cell in the compute data cube derived from the input base cuboid. Each row is formatted like input:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 


## The single-tuple optimization

Consider the naive way of recursive implementation of the BUC algorithm, you will notice that it uses several recursive calls to compute all the derived results from an input that consists of only one tuple. This is certainly a waste of computation. 

For example, if we are asked to compute the cube given the following input

 B | C | M 
---|---|---
 1 | 2 | 100

We can immmediately output the following, **without** using any recursive calls. 

<pre>
1    2    100
*    2    100
1    *    100
*    *    100
</pre>



** Note: For lab-2, you are allowed to use only two libraries, i.e., pandas, and numpy.** 

In [1]:
import pandas as pd
import numpy as np

In [2]:
##============================================================
# Data file format: 
# * tab-delimited input file
# * 1st line: dimension names and the last dimension is assumed to be the measure
# * rest of the lines: data values.

def read_data(filename):
    df = pd.read_csv(filename, sep='\t')
    return (df)
      
# helper functions
def project_data(df, d):
    # Return only the d-th column of INPUT
    return df.iloc[:, d]

def select_data(df, d, val):
    # SELECT * FROM INPUT WHERE input.d = val
    col_name = df.columns[d]
    return df[df[col_name] == val]

def remove_first_dim(df):
    # Remove the first dim of the input
    return df.iloc[:, 1:]

def slice_data_dim0(df, v):
    # syntactic sugar to get R_{ALL} in a less verbose way
    df_temp = select_data(df, 0, v)
    return remove_first_dim(df_temp)

In [3]:
def buc_rec_optimized(df):# do not change the heading of the function
    temp_list = []
    column_name = input_data.columns.values
    result = pd.DataFrame(columns = column_name)
    help_buc_fuc(df, temp_list, result)
    return result
    

def help_buc_fuc(df, temp_list, result):
    dim = df.shape[1]
    if dim == 1:
        temp_list.append(sum(project_data(df, 0)))
        length = len(result)
        # insert a row
        result.loc[length] = temp_list
    else:
        copy = temp_list[:]
        dims_value_set = set(project_data(input_data, 0))
        for i in dims_value_set:
            temp_list = copy[:]
            temp_list.append(i)
            rest = slice_data_dim0(df, i)
            help_buc_fuc(rest, temp_list, result)
        rest = remove_first_dim(df)
        temp_list = copy[:]
        temp_list.append('ALL')
        help_buc_fuc(rest, temp_list, result)


    
    

In [115]:
# def buc_rec_optimized(df):  # do not change the heading of the function
#     header = list(df)
#     result = pd.DataFrame(columns=header)
#     my_buc_rec_optimized(df, [], result)
#     return result

# def my_buc_rec_optimized(df, pre, result):  # help function
# #     print("resultshi ",result)
#     if df.shape[1] == 1:
#         pre.append(sum(helper.project_data(df, 0)))
#         result.loc[len(result)] = pre
#         print(pre)
#     else:
#         vals = set(helper.project_data(df, 0).values)
# #         print(vals,pre)
#         pre_copy = pre[:]
#         for val in vals:
#             pre = pre_copy[:]
#             sub_data = helper.slice_data_dim0(df, val)
#             pre.append(val)
#             print(pre, val)
#             my_buc_rec_optimized(sub_data, pre, result)

#         sub_data = helper.remove_first_dim(df)
#         pre = pre_copy[:]
#         pre.append("ALL")
#         print(pre)
#         my_buc_rec_optimized(sub_data, pre, result)
def buc_single(data, column):
    length = column - 1
    print(length)
    result = []
    iter_time = 2**length
    for i in range(iter_time):
        copy = [i for i in data]
        for j in range(length):
            if (i % (2 ** (length - j)) > (2 ** (length - j - 1)  - 1)):
                copy[j] = 'ALL'
        result.append(copy)
    return result

        


In [116]:
## You can test your implementation using the following code...

import helper
import submission as submission
input_data = read_data('./asset/a_.txt')
# print(input_data.shape)
# print(list(input_data.loc[0]))
buc_single(list(input_data.loc[0]), input_data.shape[1])
# output = submission.buc_rec_optimized(input_data)

# print(output)


2


[[1, 2, 100], [1, 'ALL', 100], ['ALL', 2, 100], ['ALL', 'ALL', 100]]